In [91]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [92]:
product_type = "frozen"
num_time_periods = 365

In [93]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [94]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [95]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [96]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [97]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [98]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [100]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [101]:
# TODO: Remove sundays
initial_inventory = {}
for item in demand.keys():
    initial_inventory[item] = 0
    for t in time_indexes:
        if(t%7!=6):
            if(sum(list(demand[item].values())[:t+1]) > maximum_order_quantity_tu[item] * cu_per_tu[item] * (t+1)):
                initial_inventory[item] += sum(list(demand[item].values())[:t+1]) - (maximum_order_quantity_tu[item] * cu_per_tu[item] * (t+1))


In [104]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_indexes, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, t] for item in items for t in time_indexes)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, t] for item in items for t in time_indexes)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[t] for t in time_indexes)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, t] for item in items for t in time_indexes)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for t in time_indexes:
        if(t==0):
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                +
                initial_inventory[item]
                -
                storage_used[item, t]
                -
                clearances[item, t]
                >=
                demand[item][t]
                
            )
        else:
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                +
                storage_used[item, t-1]
                -
                storage_used[item, t]
                -
                clearances[item, t]
                >=
                demand[item][t]
            )

# inventory volume constraint
for t in time_indexes:
    if(t==0):
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] * 
                (
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )
    else:
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] *
                (
                    storage_used[item,t-1]
                    +
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )

# min/max constraints (linking too)
for item in demand.keys():
    for t in time_indexes:
        m.addConstr(
            orders[item, t]
            >=
            minimum_order_quantity_tu[item] * ordered_boolean[item, t]
        )
        if((not math.isnan(maximum_order_quantity_tu[item])) and t%7!=6):
            m.addConstr(orders[item, t] <= maximum_order_quantity_tu[item] * ordered_boolean[item, t])

for item in demand.keys():
    for t in time_indexes:
        life = shelf_life[item]
        if(t >= life):
            m.addConstr(
                clearances[item, t]
                >=
                gp.quicksum(
                    cu_per_tu[item] * orders[item, t1] for t1 in time_indexes[:t-life]
                )
                -
                gp.quicksum(
                    demand[item][t1] for t1 in time_indexes[:t]
                )
                -
                gp.quicksum(
                    clearances[item, t1] for t1 in time_indexes[:t-1]
                )
            )

# sunday constraint
for item in demand.keys():
    for t in time_indexes:
        if(t%7==6):
            m.addConstr(orders[item, t] == 0)
            m.addConstr(ordered_boolean[item, t] == 0)
            m.addConstr(clearances[item, t] == 0)
            m.addConstr(storage_used[item, t] == storage_used[item, t-1])

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 422793 rows, 608751 columns and 1235165 nonzeros
Model fingerprint: 0x1fc74fe8
Variable types: 0 continuous, 608751 integer (152097 binary)
Coefficient statistics:
  Matrix range     [2e-04, 5e+02]
  Objective range  [1e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 242055 rows and 314852 columns
Presolve time: 0.12s

Explored 0 nodes (0 simplex iterations) in 0.37 seconds (0.35 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [90]:
orders

{('468a73f3', 0): <gurobi.Var C0 (value 58.0)>,
 ('468a73f3', 1): <gurobi.Var C1 (value 50.0)>,
 ('468a73f3', 2): <gurobi.Var C2 (value 87.0)>,
 ('468a73f3', 3): <gurobi.Var C3 (value -0.0)>,
 ('468a73f3', 4): <gurobi.Var C4 (value 128.0)>,
 ('468a73f3', 5): <gurobi.Var C5 (value 32.0)>,
 ('3bd76e22', 0): <gurobi.Var C6 (value 1.0)>,
 ('3bd76e22', 1): <gurobi.Var C7 (value 1.0)>,
 ('3bd76e22', 2): <gurobi.Var C8 (value 1.0)>,
 ('3bd76e22', 3): <gurobi.Var C9 (value -0.0)>,
 ('3bd76e22', 4): <gurobi.Var C10 (value -0.0)>,
 ('3bd76e22', 5): <gurobi.Var C11 (value -0.0)>,
 ('84293966', 0): <gurobi.Var C12 (value 3.0)>,
 ('84293966', 1): <gurobi.Var C13 (value 3.0)>,
 ('84293966', 2): <gurobi.Var C14 (value 3.0)>,
 ('84293966', 3): <gurobi.Var C15 (value -0.0)>,
 ('84293966', 4): <gurobi.Var C16 (value -0.0)>,
 ('84293966', 5): <gurobi.Var C17 (value 1.0)>,
 ('13541e66', 0): <gurobi.Var C18 (value 13.0)>,
 ('13541e66', 1): <gurobi.Var C19 (value 38.0)>,
 ('13541e66', 2): <gurobi.Var C20 (v